## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [370]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        # Verificación en el constructor
        if len(codigo) != 4 or not codigo.isdigit():
            raise ValueError("El código debe ser de 4 dígitos.")
        if len(nombre) < 1 or len(nombre) > 100:
            raise ValueError("El nombre debe tener entre 1 y 100 caracteres.")
        if len(tipo) > 20:
            raise ValueError("El tipo debe tener máximo 20 caracteres.")
        
        self.codigo = codigo
        self.nombre = nombre
        self.tipo = tipo
        # Utilizamos las propiedades para asignar los valores iniciales
        self._precio = None
        self._cantidad = None
        self.precio = precio  # Llama a la propiedad precio
        self.cantidad = cantidad  # Llama a la propiedad cantidad

    @property
    def precio(self):
        return self._precio

    @precio.setter
    def precio(self, valor):
        if valor < 10 or valor > 10000:
            print("Valor de precio inválido, se mantiene el anterior.")
        else:
            self._precio = valor

    @property
    def cantidad(self):
        return self._cantidad

    @cantidad.setter
    def cantidad(self, valor):
        if valor < 0 or valor > 1000:
            print("Cantidad inválida, se mantiene la anterior.")
        else:
            self._cantidad = valor

    def valorTotal(self):
        return self.precio * self.cantidad



In [371]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

Valor de precio inválido, se mantiene el anterior.
Cantidad inválida, se mantiene la anterior.
Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [372]:
class Oferta:
    def __init__(self, descripcion='Oferta sin descripción', codigos=[], tipos=[]):
        if len(descripcion) < 1 or len(descripcion) > 100:
            raise ValueError("La descripción debe tener entre 1 y 100 caracteres.")
        self.descripcion = descripcion
        self.codigos = codigos
        self.tipos = tipos
    
    def esAplicable(self, producto, cantidad):
        return producto.codigo in self.codigos or producto.tipo in self.tipos

class OfertaDescuento(Oferta):
    def __init__(self, descuento, *args, **kwargs):
        super().__init__(*args, **kwargs)
        if descuento < 0 or descuento > 100:
            raise ValueError("El descuento debe ser un porcentaje entre 0 y 100.")
        self.descuento = descuento

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (producto.precio * cantidad) * (self.descuento / 100)
        return 0

class Oferta2x1(Oferta):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            return (cantidad // 2) * producto.precio
        return 0


In [373]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [374]:
class Catalogo:
    def __init__(self):
        self.productos = []
        self.ofertas = []

    @classmethod
    def leer(cls, archivo):
        import csv
        catalogo = cls()
        with open(archivo, 'r') as f:
            reader = csv.reader(f)
            next(reader)
            for row in reader:
                codigo, nombre, precio, tipo, cantidad = row
                producto = Producto(codigo, nombre, float(precio), tipo, int(cantidad))
                catalogo.agregar(producto)
        return catalogo

    def guardar(self, archivo):
        import csv
        with open(archivo, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Código', 'Nombre', 'Precio', 'Tipo', 'Cantidad'])
            for producto in self.productos:
                writer.writerow([producto.codigo, producto.nombre, producto.precio,
                                 producto.tipo, producto.cantidad])

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None

    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def calcularDescuento(self, producto, cantidad):
        max_descuento = 0
        for oferta in self.ofertas:
            descuento = oferta.calcularDescuento(producto, cantidad)
            if descuento > max_descuento:
                max_descuento = descuento
        return max_descuento

    @property
    def cantidadProductos(self):
        return len(self.productos)

    @property
    def cantidadUnidades(self):
        return sum(p.cantidad for p in self.productos)

    @property
    def valorTotal(self):
        return sum(p.precio * p.cantidad for p in self.productos)

    def vender(self, producto, cantidad):
        if cantidad <= producto.cantidad:
            producto.cantidad -= cantidad

    def informe(self):
        tipos = sorted(set(p.tipo for p in self.productos))
        precio_promedio = self.valorTotal / self.cantidadUnidades if self.cantidadUnidades > 0 else 0
        
        informe = f"""Informe de Catalogo
====================
Cantidad de productos: {self.cantidadProductos}
Cantidad de unidades: {self.cantidadUnidades}
Precio Promedio: {precio_promedio:.2f}
Valor total: {self.valorTotal:.2f}
Tipos de productos: {', '.join(tipos)}
Ofertas:
"""
        for oferta in self.ofertas:
            if isinstance(oferta, Oferta2x1):
                informe += "Oferta 2x1\n"
            else:
                informe += f"{oferta.descripcion}\n"
                
        return informe


In [375]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')

assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [376]:
import re
class Cliente:
    def __init__(self, nombre_completo, cuit):
        nombre_completo_separado = nombre_completo.split()
        self._nombre = self.validacion_nombre(nombre_completo_separado[0])
        self._apellido = self.validacion_apellido(' '.join(nombre_completo_separado[1:])) if len(nombre_completo_separado) > 1 else ''
        self._cuit = self.validacion_cuit(cuit)
    @property
    def nombre(self):
        return f"{self._nombre} {self._apellido}".strip()
    @nombre.setter
    def nombre(self, nuevo_nombre):
        nombre_separado = nuevo_nombre.split()
        if len(nombre_separado) == 0 or nuevo_nombre.strip() == '':
            return 
        try:
            self._nombre = self.validacion_nombre(nombre_separado[0])
            self._apellido = self.validacion_apellido(' '.join(nombre_separado[1:])) if len(nombre_separado) > 1 else ''
        except ValueError:
            pass
    @property
    def cuit(self):
        return self._cuit
    @cuit.setter
    def cuit(self, nuevo_cuit):
        try:
            self._cuit = self.validacion_cuit(nuevo_cuit)
        except ValueError:
            pass

    def validacion_nombre(self, nombre):
        if 1 <= len(nombre) <= 100:
            return nombre
        else:
            raise ValueError("Nombre invalido, debe tener de 1 a 100 caracteres")

    def validacion_apellido(self, apellido):
        if 1 <= len(apellido) <= 100:
            return apellido
        else:
            raise ValueError("Apellido invalido, debe tener de 1 a 100 caracteres")

    def validacion_cuit(self, cuit):
        if re.match(r'^\d{2}-\d{8}-\d$', cuit):
            return cuit
        else:
            raise ValueError("El cuit debe tener el siguiente formato: XX-XXXXXXXX-X.")


In [377]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [378]:
from datetime import datetime
class Factura:
    
    contador = 100 
    @classmethod
    def ultimaFactura(c, num):
        c.contador = num
    @classmethod
    def nuevoNumero(c):
        c.contador += 1
        return c.contador

    def __init__(self, catalogo, cliente):
        self.vendidos = []
        self.cliente = cliente
        self.catalogo = catalogo
        self.fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.numero = Factura.nuevoNumero()
        
    @property
    def cantidadProductos(self):
        return len(set(producto for producto, _ in self.vendidos))
    @property
    def cantidadUnidades(self):
        return sum(cantidad for _, cantidad in self.vendidos)
    @property
    def subtotal(self):
        return sum(producto.precio * cantidad for producto, cantidad in self.vendidos)

    @property
    def descuentos(self):
        total_descuentos = 0
        for producto, cantidad in self.vendidos:
            total_descuentos += self.catalogo.calcularDescuento(producto, cantidad)
        return total_descuentos

    @property
    def total(self):
        return self.subtotal - self.descuentos

    def agregar(self, producto, cantidad):
        if cantidad > 0 and producto.cantidad >= cantidad:
            for i in range(len(self.vendidos)):
                if self.vendidos[i][0] == producto:
                    nuevo_cantidad = self.vendidos[i][1] + cantidad
                    if nuevo_cantidad <= producto.cantidad:  
                        producto.cantidad -= cantidad 
                        self.vendidos[i] = (producto, nuevo_cantidad)
                        return
            
            self.vendidos.append((producto, cantidad))
            producto.cantidad -= cantidad 

    def imprimir(self):
        factura = f"Factura: {self.numero}\n"
        factura += f"Fecha  : {self.fecha}\n"
        factura += f"Cliente: {self.cliente.nombre} ({self.cliente.cuit})\n\n"
        dic_productos = {}
        for producto, cantidad in self.vendidos:
            if producto in dic_productos:
                dic_productos[producto] += cantidad
            else:
                dic_productos[producto] = cantidad

        for producto, cantidad in dic_productos.items():
            subtotal_producto = producto.precio * cantidad
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            factura += f"{cantidad} {producto.nombre:<30} x ${producto.precio:.2f} = ${subtotal_producto:.2f}\n"
        
        if descuento > 0:
            if producto.codigo in ['0001', '0003']:
                factura += f"Descuento 10% - ${descuento:.2f}\n"
            if producto.tipo == 'galleta':
                factura += f"Oferta 2x1 - ${descuento:.2f}\n"

        factura += f"\n{' ' * 41}Subtotal:   ${self.subtotal:.2f}\n"
        factura += f"{' ' * 41}Descuentos: ${self.descuentos:.2f}\n"
        factura += f"{' ' * 41}--------------------\n"
        factura += f"{' ' * 41}TOTAL:      ${self.total:.2f}\n"

        return factura


In [379]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


In [380]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #
catalogo
# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
